In [1]:
from datasets import load_dataset

config = ['Accounting', 'Agriculture', 'Architecture_and_Engineering', 'Art', 'Art_Theory', 'Basic_Medical_Science', 'Biology', 'Chemistry', 'Clinical_Medicine', 'Computer_Science', 'Design', 'Diagnostics_and_Laboratory_Medicine', 'Economics', 'Electronics', 'Energy_and_Power', 'Finance', 'Geography', 'History', 'Literature', 'Manage', 'Marketing', 'Materials', 'Math', 'Mechanical_Engineering', 'Music', 'Pharmacy', 'Physics', 'Psychology', 'Public_Health', 'Sociology']

ds = load_dataset("MMMU/MMMU",config[0])

In [2]:
gpu_index = 6
training_gpu = "cuda:"+str(gpu_index)

In [3]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True)
model.to(f"cuda:{gpu_index}")


Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlavaNextForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn

In [4]:
from baukit import Trace

In [5]:
torch.set_grad_enabled(False)

In [6]:
import os

In [7]:
pwd

'FINAL'

In [8]:
layer = 20
file = f"/SAE_imagetext_layer{layer}_models_longrun/" +os.listdir(f"/SAE_imagetext_layer{layer}_models_longrun/")[0]

In [9]:
import torch
import torch.nn as nn
class SAE(nn.Module):
    def __init__(self):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(4096,int(8*4096),bias=True) 
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(int(8*4096), 4096,bias=True) 
        self.initialize_weights()
    #Biases are initialized to zero, 
    def initialize_weights(self):
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)
        #fun trick, gaussians in high dimensions are uniform "soap bubbles"
        W_d = torch.randn(self.fc2.weight.size())
        W_d = W_d / W_d.norm(dim=1, keepdim=True) * 0.1  # Adjust 0.1 as needed
        self.fc2.weight.data = W_d
        self.fc1.weight.data = W_d.t()
    def forward(self, x):
        features = self.relu1(self.fc1(x))
        x_hat = self.fc2(features)
        return x_hat,features 
sae = SAE()
sae = nn.DataParallel(sae, device_ids=[gpu_index])
sae = sae.cuda(device=gpu_index)
sae.load_state_dict(torch.load(file))
def forward_ablate(text_input,image,tokens):
        intervention = intervene
        inp = processor(text_input,image, return_tensors="pt").to(training_gpu)
        for i in range(tokens):    
            with Trace(model.language_model.model.layers[layer-1],edit_output=intervention) as ret:
                out = model(**inp)
            logits = out.logits[0][-1]
            inp['input_ids'] = torch.tensor(np.array(list(inp.input_ids.detach().cpu().numpy()[0])+[torch.argmax(logits).detach().cpu().numpy()])).unsqueeze(0).to(training_gpu)
            inp['attention_mask'] = torch.ones((inp['attention_mask'].size(0),inp['attention_mask'].size(1)+1)).int().to(training_gpu)
        return inp['input_ids'][0]
    
def generate_ablate(input_text,imp_image,num_tokens):
    ids = forward_ablate(input_text,imp_image,num_tokens)
    return processor.decode(ids,skip_special_tokens=True)

def intervene(output):
    meow = output[0].squeeze(0).float()
    meow = sae(meow.unsqueeze(0)*8)[0].squeeze(0)/8
    return (meow.unsqueeze(0).half(),output[1])

/tmp/ipykernel_3514303/995103250.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sae.load_state_dict(torch.load(file))


In [10]:
def forward_ablate(text_input,image,tokens):
        intervention = intervene
        inp = processor(text_input,image.resize((336,336)), return_tensors="pt").to(training_gpu)
        for i in range(tokens):    
            with Trace(model.language_model.model.layers[layer-1],edit_output=intervention) as ret:
                out = model(**inp)
            logits = out.logits[0][-1]
            inp['input_ids'] = torch.tensor(np.array(list(inp.input_ids.detach().cpu().numpy()[0])+[torch.argmax(logits).detach().cpu().numpy()])).unsqueeze(0).to(training_gpu)
            inp['attention_mask'] = torch.ones((inp['attention_mask'].size(0),inp['attention_mask'].size(1)+1)).int().to(training_gpu)
        return inp['input_ids'][0]
    
def generate_ablate(input_text,imp_image,num_tokens):
    ids = forward_ablate(input_text,imp_image,num_tokens)
    return processor.decode(ids,skip_special_tokens=True)

def intervene(output):
    meow = output[0].squeeze(0).float()
    meow[:1183] = sae(meow[:1183].unsqueeze(0)*8)[0].squeeze(0)/8
    return (meow.unsqueeze(0).half(),output[1])

In [11]:
import ast

In [12]:

def load_q(i):
    q = '[INST] \<image>\n '+ds[i]['question'].replace('<image 1>','the image')
    options = ast.literal_eval(ds[i]['options'])
    letters = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N']
    used_letters = []
    for j in range(len(options)):
        q = q+f"\n {letters[j]}: "+options[j]
        used_letters.append(letters[j])
    
    q = q + f"\n Please Answer only answer with one of {used_letters}. Do not provide any explanation. [/INST]"
    images = []
    for j in range(1,8):
        if ds[i]['image_'+str(j)] is not None:
            images.append(ds[i]['image_'+str(j)])
    return q,images[0], ds[i]['answer'], len(options)

In [13]:
import warnings
from transformers import logging

# Turn off warnings from the transformers library
logging.set_verbosity_error()

# If you want to suppress all Python warnings, including those from other libraries
warnings.filterwarnings('ignore')


In [14]:
import numpy as np

In [15]:
from tqdm import tqdm
cnt = 0
tot = 0
chance = 0

for l in range(len(config)):
    ds = load_dataset("MMMU/MMMU",config[l])
    ds = ds['validation']
    pbar = tqdm(range(30))
    for i in pbar:
        prompt, images, answer,num_answers = load_q(i)
        try:
            inputs = processor(images, prompt, return_tensors="pt").to("cuda:0")
            
            # autoregressively complete prompt
            
            
            # autoregressively complete prompt
            output = generate_ablate(prompt,images,1)
            
            rep = output[-1]
            if rep == answer:
                cnt+=1
                
            tot +=1
            chance+=1/num_answers
            pbar.set_description("MODEL: "+str(round(cnt/tot, 4)) + " Chance: "+ str(round(chance/tot, 4)))
        except KeyboardInterrupt:
            break
        except:
            print("sadge")
        
print(cnt/tot)

MODEL: 0.3014 Chance: 0.258:  47%|██████▌       | 14/30 [00:06<00:07,  2.22it/s]

sadge


MODEL: 0.3899 Chance: 0.2551:  33%|████▎        | 10/30 [00:04<00:09,  2.21it/s]

sadge


MODEL: 0.3804 Chance: 0.2545:  47%|██████       | 14/30 [00:06<00:07,  2.21it/s]

...

sadge


MODEL: 0.3366 Chance: 0.2468:  67%|████████▋    | 20/30 [00:11<00:05,  1.78it/s]

sadge


MODEL: 0.3433 Chance: 0.247: 100%|██████████████| 30/30 [00:20<00:00,  1.44it/s]

0.3433333333333333


In [21]:

            
# autoregressively complete prompt
output = generate_ablate(prompt,images,1)

rep = output[-1]
if rep == answer:
    cnt+=1
if rep not in ['A','B','C','D','E','F','G','H','I','J','K','L','M','N']:
    print("error")

NameError: name 'np' is not defined

In [35]:
print(output)

[INST] Donna Donie, CFA, has a client who believes the common stock price of TRT Materials (currently $58 per share) could move substantially in either direction in reaction to an expected court decision involving the company. The client currently owns no TRT shares, but asks Donie for advice about implementing a strangle strategy to capitalize on the possible stock price movement. A strangle is a portfolio of a put and a call with different exercise prices but the same expiration date. Donie gathers the TRT option-pricing data: 
 
 Calculate, at expiration for long strangle strategy, the Maximum possible loss per share.
 A: $9.00
 B: $5.00
 C: the Maximum  possible loss is unlimited
 Please Answer only answer with one of ['A', 'B', 'C']. Do not provide any explanation. [/INST]5 25,


In [24]:
output

"[INST] Donna Donie, CFA, has a client who believes the common stock price of TRT Materials (currently $58 per share) could move substantially in either direction in reaction to an expected court decision involving the company. The client currently owns no TRT shares, but asks Donie for advice about implementing a strangle strategy to capitalize on the possible stock price movement. A strangle is a portfolio of a put and a call with different exercise prices but the same expiration date. Donie gathers the TRT option-pricing data: \n \n Calculate, at expiration for long strangle strategy, the Maximum possible loss per share.\n A: $9.00\n B: $5.00\n C: the Maximum  possible loss is unlimited\n Please Answer only answer with one of ['A', 'B', 'C']. Do not provide any explanation. [/INST]5"

In [21]:
import numpy as np

In [76]:
processor.pad_token_id=None

In [ ]:
options

In [58]:
options[7]

'Visual elements'

In [10]:
qa
print()

"Which of the following best explains the overall trend shown in the <image>?\n A: [\n B: '\n C: M\n D: M"

Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlavaNextForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn

In [9]:
!pip install --upgrade jinja2

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 KB 3.4 MB/s eta 0:00:0000:01
